In [50]:
import numpy as np
import time 

def MotionTrack(f1, f2):
    
    #pythagorean distance between two frames
    f1_32 = np.float32(f1)
    f2_32 = np.float32(f2)
    diff32 = f1_32 - f2_32
    norm32 = np.sqrt(diff32[:,:,0]**2 + diff32[:,:,1]**2 + diff32[:,:,2]**2)/np.sqrt(255**2 + 255**2 + 255**2)
    motion = np.uint8(norm32*255)
    return motion


#Maybe try putting this on its own?
def move(begin):
    #timecap = 60  #Used if you want to cap the movement time recorded, in seconds
    st = 0
    
    while (st > 8):
        global en 
        en = (int(time.time()-begin))
        _, frame3 = vcap.read()
        _, frame1 = vcap.read()
        _, frame2 = vcap.read()
        rows, cols, _ = np.shape(frame3)
        dist = MotionTrack(frame1, frame3)
        frame1 = frame2
        frame2 = frame3
        mod = cv2.GaussianBlur(dist, (9,9), 0)
        _, thresh = cv2.threshold(mod, 100, 255, 0)
        _, st = cv2.meanStdDev(mod)
    return en
   #print(end)



def punish(begin):
   # timecap = 60  Used if you want to cap the movement time recorded, if not -> while True: 
    mask = 0
    while(cv2.countNonZero(mask) == 0):
    
        en2 = (int(time.time()-begin))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
        lower_dark = np.array([0,0,0]) 
        upper_dark = np.array([75,80,4])
        mask = cv2.inRange(hsv, lower_dark, upper_dark)
        #print(en2)
    return en2





In [51]:
import cv2
from datetime import datetime

#Variable initialization

sdThresh = 8 # REMEMBER THIS, changes sensitivity to motion 

global mask
global en2
global en

font = cv2.FONT_HERSHEY_SIMPLEX
vcap = cv2.VideoCapture("rtsp://172.16.182.217:554/chID=2&streamType=Sub")
_, frame1 = vcap.read()
_, frame2 = vcap.read()
fps = vcap.get(cv2.CAP_PROP_FPS)   
timestamps = [vcap.get(cv2.CAP_PROP_POS_MSEC)]
fmt = "%Y-%m-%d %H:%M:%S %Z%z"
birdbad = 0
birdmove = 0


while(1):
    ret, frame = vcap.read()
    #cv2.imshow('Raw', frame)
    cv2.waitKey(1)
    _, frame3 = vcap.read()
    rows, cols, _ = np.shape(frame3)
    dist = MotionTrack(frame1, frame3)
    
    frame1 = frame2
    frame2 = frame3
    
    mod = cv2.GaussianBlur(dist, (9,9), 0)
    _, thresh = cv2.threshold(mod, 100, 255, 0)
    _, stDev = cv2.meanStdDev(mod)

    
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
    lower_dark = np.array([0,0,0]) 
    upper_dark = np.array([75,80,4])
    mask = cv2.inRange(hsv, lower_dark, upper_dark)
       
        # Bird count testing 
    if(cv2.countNonZero(mask) == 0):
        birdbad = birdbad+1
        now = datetime.now()
        begin = time.time()
        end = punish(begin)
        print("Bird punished for " + str(end) + ' seconds starting at ' + str(now.strftime(fmt)))
        print("Bird punished " + str(birdbad) + ' times')
        print()

        
vcap.release()

KeyboardInterrupt: 

In [ ]:
  ##PUT WITH MAIN CODE
    # Bird movement
    if stDev > sdThresh:
        birdmove = birdmove+1
        now = datetime.now()
        begin = time.time()
        end = move(begin)
        print("Bird moved for " + str(end) + ' seconds starting at ' + str(now.strftime(fmt)))
        print("Bird moved " + str(birdmove) + ' times')
        print()


In [ ]:
     #Bird punishment
    if(cv2.countNonZero(mask) == 0):
        birdbad = birdbad+1
        now = datetime.now()
        begin = time.time()
        end = punish(begin)
        print("Bird punished for " + str(end) + ' seconds starting at ' + str(now.strftime(fmt)))
        print("Bird punished " + str(birdbad) + ' times')
        print()


In [ ]:
#Destroy window of live feed if necessary 
cv2.destroyWindow('Raw')